In [ ]:
# from datetime import datetime 
# import pandas as pd
# from utils import simulator_period,save_sim_resuls
# from subjects.corr_coef import CorrCoeff,TrendFunc
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
# casino_fig=pd.read_csv('casino_fig.csv', parse_dates = ['Date'])
# casino_fig['week_day'] = casino_fig['Date'].dt.dayofweek

In [ ]:
# casino_fig.head()

#### Heads прогнозирование

In [ ]:

# casino_fig[:].plot(x="Date", y ="Heads")


### Разделяем на тренингувую и тестовую выборку

In [ ]:
# casino_fig_train = casino_fig[casino_fig['Date'] < datetime(2024,11,1) ]
# casino_fig_test = casino_fig[casino_fig['Date'] >= datetime(2024,11,1) ]

In [ ]:
# casino_fig_test.head()

In [ ]:
# casino_fig_train.tail()

### Вычисляем Тренд

In [ ]:
# coefficients = np.polyfit(casino_fig_train.index,casino_fig_train['Heads'], 1)
# print(coefficients)

In [ ]:
# poly_1 = np.poly1d(coefficients)

In [ ]:
# casino_fig_train['trend_heads']= poly_1(casino_fig_train.index)

In [ ]:
# casino_fig_train.plot(x="Date", y ="trend_heads")

### Очищаем от тренда

In [ ]:
# casino_fig_train['heads_center_and_clear']=casino_fig_train['Heads']-casino_fig_train['trend_heads']

In [ ]:
# casino_fig_train.plot(x="Date", y ="Heads")
# casino_fig_train.plot(x="Date", y ="heads_center_and_clear")

In [ ]:
# heads_train_df =casino_fig_train.groupby('week_day')['heads_center_and_clear'].agg([ 'mean','std'])

In [ ]:
# heads_train_df

### Добавляем тренд в тестовую выборку (для удобства его учёта при прогнозировании)

In [ ]:
# casino_fig_test['trend_heads']= poly_1(casino_fig_test.index)

### Делаем прогноз

In [ ]:
# predict = []
# predict_3_std_plus = []
# predict_3_std_minus = []
# for curr_ind in casino_fig_test.index:
#     week_day = casino_fig_test['week_day'].loc[curr_ind]
#     trend = casino_fig_test['trend_heads'].loc[curr_ind]
#     pred = trend + heads_train_df['mean'].loc[week_day]
#     predict.append(pred)
#     predict_3_std_plus.append(pred+3*heads_train_df['std'].loc[week_day])
#     predict_3_std_minus.append(pred-3*heads_train_df['std'].loc[week_day])
    

In [ ]:
# casino_fig_test['predict'] = predict
# casino_fig_test['predict_3_std_plus'] = predict_3_std_plus
# casino_fig_test['predict_3_std_minus'] = predict_3_std_minus

In [ ]:
# plt.plot(casino_fig_test['Date']  ,casino_fig_test['Heads'] )
# plt.plot(casino_fig_test['Date'],casino_fig_test['predict'],color='green' )
# plt.fill_between(casino_fig_test['Date'], casino_fig_test['predict_3_std_minus'], casino_fig_test['predict_3_std_plus'], alpha=0.5, color='orange')
# plt.tick_params(axis='x', labelrotation=45)